In [1]:
import os
import fnmatch
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import regex as re
import operator
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split  
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import pickle
from nltk.corpus import stopwords
path = r'\Users\ilans\Documents\Studies\Semester 5\Final Project\op_spam_v1.4'

label = []

configfiles = [os.path.join(subdir,f)
for subdir, dirs, files in os.walk(path)
    for f in fnmatch.filter(files, '*.txt')]

for f in configfiles:
    c = re.search('(trut|deceptiv)\w',f)
    label.append(c.group())

labels = pd.DataFrame(label, columns = ['Labels'])

review = []
directory =os.path.join(r'\Users\ilans\Documents\Studies\Semester 5\Final Project\op_spam_v1.4')
for subdir,dirs ,files in os.walk(directory):
   # print (subdir)
    for file in files:
        if fnmatch.filter(files, '*.txt'):
            f=open(os.path.join(subdir, file),'r')
            a = f.read()
            review.append(a)
            
reviews = pd.DataFrame(review, columns = ['HotelReviews'])
result = pd.merge(reviews, labels,right_index=True,left_index = True)


result['HotelReviews'] = result['HotelReviews'].map(lambda x: x.lower())
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')
result['review_without_stopwords'] = result['HotelReviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
result.head().append(result.tail())
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def pos(review_without_stopwords):
    return TextBlob(review_without_stopwords).tags
os = result.review_without_stopwords.apply(pos)
os1 = pd.DataFrame(os)
os1['pos'] = os1['review_without_stopwords'].map(lambda x:" ".join(["/".join(x) for x in x ]) )
result = result = pd.merge(result, os1,right_index=True,left_index = True)
result.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ilans\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ilans\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ilans\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,HotelReviews,Labels,review_without_stopwords_x,review_without_stopwords_y,pos
0,we stayed at the schicago hilton for 4 days an...,deceptive,stayed schicago hilton 4 days 3 nights confere...,"[(stayed, VBN), (schicago, JJ), (hilton, NN), ...",stayed/VBN schicago/JJ hilton/NN 4/CD days/NNS...
1,hotel is located 1/2 mile from the train stati...,deceptive,hotel located 1/2 mile train station quite hik...,"[(hotel, NN), (located, VBD), (1/2, CD), (mile...",hotel/NN located/VBD 1/2/CD mile/NN train/NN s...
2,i made my reservation at the hilton chicago be...,deceptive,made reservation hilton chicago believing goin...,"[(made, VBN), (reservation, NN), (hilton, NN),...",made/VBN reservation/NN hilton/NN chicago/NN b...
3,"when most people think hilton, they think luxu...",deceptive,"people think hilton, think luxury. know did. w...","[(people, NNS), (think, VBP), (hilton, NN), (t...",people/NNS think/VBP hilton/NN think/VBP luxur...
4,my husband and i recently stayed stayed at the...,deceptive,husband recently stayed stayed hilton chicago ...,"[(husband, NN), (recently, RB), (stayed, VBD),...",husband/NN recently/RB stayed/VBD stayed/JJ hi...


In [2]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=5)

tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True, sublinear_tf=False)

X_train_tf = tf_vect.fit_transform(review_train)
X_test_tf = tf_vect.transform(review_test)

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_
svc_param_selection(X_train_tf,label_train,5)
clf = svm.SVC(C=10,gamma=0.001,kernel='linear')
clf.fit(X_train_tf,label_train)
pred = clf.predict(X_test_tf)
with open('vectorizer.pickle', 'wb') as fin:
    pickle.dump(tf_vect, fin)
with open('mlmodel.pickle','wb') as f:
    pickle.dump(clf,f)
    
pkl = open('mlmodel.pickle', 'rb')
clf = pickle.load(pkl)   
vec = open('vectorizer.pickle', 'rb')
tf_vect = pickle.load(vec)
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

print(metrics.accuracy_score(label_test, pred))

0.9


In [3]:
result['review_without_stopwords_y'][0]

[('stayed', 'VBN'),
 ('schicago', 'JJ'),
 ('hilton', 'NN'),
 ('4', 'CD'),
 ('days', 'NNS'),
 ('3', 'CD'),
 ('nights', 'NNS'),
 ('conference', 'NN'),
 ('say', 'NN'),
 ('normally', 'RB'),
 ('easy', 'JJ'),
 ('going', 'VBG'),
 ('amenities', 'NNS'),
 ('cleanliness', 'NN'),
 ('like', 'IN'),
 ('however', 'RB'),
 ('experience', 'IN'),
 ('hilton', 'NN'),
 ('awful', 'IN'),
 ('taking', 'VBG'),
 ('time', 'NN'),
 ('actually', 'RB'),
 ('write', 'JJ'),
 ('review', 'NN'),
 ('truly', 'RB'),
 ('stay', 'JJ'),
 ('hotel', 'NN'),
 ('arrived', 'JJ'),
 ('room', 'NN'),
 ('clear', 'JJ'),
 ('carpet', 'NN'),
 ('vacuumed', 'VBD'),
 ('figuered', 'VBN'),
 ('okay', 'NN'),
 ('carpet', 'NN'),
 ('saw', 'NN'),
 ('bathroom', 'NN'),
 ('although', 'IN'),
 ('bathroom', 'JJ'),
 ('superficial', 'JJ'),
 ('indicators', 'NNS'),
 ('housekeeping', 'VBG'),
 ('recently', 'RB'),
 ('cleaned', 'VBN'),
 ('i.e.', 'FW'),
 ('paper', 'NN'),
 ('band', 'NN'),
 ('across', 'IN'),
 ('toilet', 'NN'),
 ('paper', 'NN'),
 ('caps', 'NNS'),
 ('drinking

In [4]:
print (confusion_matrix(label_test, pred))



[[128  16]
 [ 16 160]]


In [5]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.89      0.89      0.89       144
       truth       0.91      0.91      0.91       176

    accuracy                           0.90       320
   macro avg       0.90      0.90      0.90       320
weighted avg       0.90      0.90      0.90       320



In [180]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=1)

X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)
print(metrics.accuracy_score(label_test, pred))

0.978125


In [228]:
print (classification_report(label_test, pred))

              precision    recall  f1-score   support

   deceptive       0.89      0.89      0.89       144
       truth       0.91      0.91      0.91       176

    accuracy                           0.90       320
   macro avg       0.90      0.90      0.90       320
weighted avg       0.90      0.90      0.90       320



In [229]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=10)
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)
print(metrics.accuracy_score(label_test, pred))

0.96875


In [230]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=42)
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)
print(metrics.accuracy_score(label_test, pred))

0.98125


In [231]:
def test_string(s):
    X_test_tf = tf_vect.transform([s])
    y_predict = clf.predict(X_test_tf)
    return y_predict
test_string("If you miss rudeness If you have nowhere to throw your money away That way ! Feel at home They'll make you look like an idiot Enjoy !!!!  ")


array(['deceptive'], dtype=object)

In [50]:
test_string("If you miss rudeness If you have nowhere to throw your money away That way ! Feel at home They'll make you look like an idiot Enjoy !!!!  ")

array(['deceptive'], dtype=object)

array(['deceptive'], dtype=object)

In [142]:
new_reviews = pd.read_excel (r'C:\\Users\\ilans\\Documents\\Studies\\Semester 5\\Final Project\\op_spam_v1.4\\hotelReviewsAll.xlsx')
new_reviews.head()


,name,title,desc,user
0,Shoreham,Booking messed up,The hotel messed up on the booking and then wa...,sarahpustilnik
1,Shoreham,Couples 1 Night Stay,"The room was ok, a little smaller than expecte...",Alive1017
2,Shoreham,Great stay,Staff was great and helpful with everything. T...,britpaco24
3,Shoreham,"Great Staff, Cleanliness could have been better!","Amazing staff, they were extremely helpful and...",mrsrush0109
4,Shoreham,"Wow, there is just no way",There is no way this is a four star accommodat...,Cruiser550427


In [143]:

to_test=[]
NR=new_reviews['desc']
test_results=[]
for i in NR:
    X_test_tf = tf_vect.transform([i])
    y_predict = clf.predict(X_test_tf)
    test_results.append(y_predict)

In [144]:
TRUE_TEST=[]
for i in test_results:
    TRUE_TEST.append(i[0])

In [145]:
new_reviews['reliability']=TRUE_TEST
new_reviews.to_excel('C:\\Users\\ilans\\Documents\\Studies\\Semester 5\\Final Project\\op_spam_v1.4\\hotelReviewsAllAnalyzed.xlsx')

In [188]:
cnt_truth=0
cnt_deceptive=0
cnt_error=0
for i in TRUE_TEST:
    if i=='truth':
        cnt_truth+=1
    if i=='deceptive':
        cnt_deceptive+=1
print(cnt_truth)
print(cnt_deceptive)


        

33084
7124


In [58]:
test_string("dsadfd")

array(['truth'], dtype=object)

In [59]:
test_string(revs[0])

array(['truth'], dtype=object)

In [222]:
text = word_tokenize("And now for something completely different")nltk.pos_tag(text)
doc = nltk(text)

# Obtain tags using pos_tag
print("The pos tags of the given text:")
for tok in doc:
    print((tok.text, tok.tag_))

print("*"*20)          
print("The universal pos tags of the given text")
print("*"*20) 
for tok in doc:
    print((tok.text, tok.pos_))

print("*"*20)
print("The extracted noun phrases is: ")
print("*"*20) 
for noun_chunk in doc.noun_chunks:
    print(noun_chunk.text)

SyntaxError: invalid syntax (<ipython-input-222-0f5b3d4293cd>, line 1)

In [149]:
print (confusion_matrix(label_test, pred))

[[165   3]
 [  3 149]]
